# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, consult the forums for more troubleshooting tips.**  

## Import Packages

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Read in an Image

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def get_a_b_of_line(line):
    """
    Returns the slope and the intercept of the line
    If the points have very close X or Y coordiantes or both, 
    returns 0,0. This filters out vertical, horizontal and short lines,
    which shows benefitial results. 
    """
    a, b = 0, 0
    threshold = 5
    if (abs(line[0] - line[2]) > threshold) and (abs(line[1] - line[3]) > threshold):
        # a = (y1 - y2)/(x1 - x2)
        # b = y1 - a * x1
        a = (line[1] - line[3])/(line[0] - line[2])
        b = line[1] - a * line[0]
    return a, b

def is_in_bounds(num, bounds):
    """
    Returns True if the number num is contained within the
    specified bounds. Bounds is defined with two parameters in the list:
    lower bound and upper bound
    """
    if num >= bounds[0] and num <= bounds[1]:
        return True
    else:
        return False
    
def get_left_and_right_lines_bounds(lines, vertices_mid):
    """
    Returns bounds for a and b on both sides of field of view. 
    The idea is that the outliers should be filtered out when we
    calcualte the average a and b in the combine_lines method. 
    With this method we return the bounds in which we recommend
    a and b to be accepted, and then we use these bounds later 
    to take into account only the lines with a and b within these
    bounds.
    """
    
    # These lists represent all a and b values on both sides
    all_left_a  = []
    all_left_b  = []
    all_right_a = []
    all_right_b = []

    # Iterate over all lines and perform calcualtions
    for line in lines:
        a, b = get_a_b_of_line(line[0])
        # Ignore the lines for which get_a_b_of_line returns 0 
        if a == 0 or b == 0:
            continue
        if line[0][0] < vertices_mid:
            # left side of the picture
            all_left_a.append(a)
            all_left_b.append(b)
        else:
            # Right side of the picture
            all_right_a.append(a)
            all_right_b.append(b)

    # Number of points in total on both sides
    left_num  = len(all_left_a)
    right_num = len(all_right_a)
        
    """
    print ("All left lines")
    for i in range(left_num):
        print(all_left_a[i], all_left_b[i])
        
    print ("All right lines")
    for i in range(right_num):
        print(all_right_a[i], all_right_b[i])
    """

    # Average a and b for both sides
    # Assume zero if no points are grabbed (left_num or right_num are zero)
    
    if left_num > 0:
        left_avg_a  = sum(all_left_a)  / left_num
        left_avg_b  = sum(all_left_b)  / left_num
    else:
        left_avg_a, left_avg_b = 0, 0
        
    if right_num > 0:    
        right_avg_a = sum(all_right_a) / right_num
        right_avg_b = sum(all_right_b) / right_num
    else:
        right_avg_a, right_avg_b = 0, 0
    
    """
    print ("Found size and averages on both sides")
    print (left_num, left_avg_a, left_avg_b, right_num, right_avg_a, right_avg_b)
    """
    
    # Empirically define factors for upper and lower bound
    fact_upper_bound = 1.2
    fact_lower_bound = 0.8
    left_bound_a  = [fact_lower_bound * left_avg_a,  fact_upper_bound * left_avg_a]
    left_bound_b  = [fact_lower_bound * left_avg_b,  fact_upper_bound * left_avg_b]
    right_bound_a = [fact_lower_bound * right_avg_a, fact_upper_bound * right_avg_a]
    right_bound_b = [fact_lower_bound * right_avg_b, fact_upper_bound * right_avg_b]
    # Sort to make sure that index 0 is lower (negative bounds!)
    left_bound_a.sort()
    left_bound_b.sort()
    right_bound_a.sort()
    right_bound_b.sort()
    
    """
    print ("Found bounds for left and right a and b")
    print (left_bound_a, left_bound_b, right_bound_a, right_bound_b)
    """
    
    return all_left_a, all_left_b, all_right_a, all_right_b, left_bound_a, left_bound_b, right_bound_a, right_bound_b

def get_average_lines(all_left_a, all_left_b, all_right_a, all_right_b, left_bound_a, left_bound_b, right_bound_a, right_bound_b, vertices):
    """
    Takes the precalculated values of all left and right a and b parameters for the lines. It also takes
    the expected bounds. It filters out only those lines (a and b values) that fall within the bounds. From
    these filtered lines, it calculates a and b of the average lines (one on each side)
    """

    # These lists represent filtered a and b values on both sides
    filtered_left_a  = []
    filtered_left_b  = []
    filtered_right_a = []
    filtered_right_b = []
    
    # Process all a and b values on the left
    for i in range(len(all_left_a)):
        a = all_left_a[i]
        b = all_left_b[i]
        # Check if within bounds
        if is_in_bounds(a, left_bound_a) or is_in_bounds(b, left_bound_b):
            filtered_left_a.append(a) 
            filtered_left_b.append(b) 
    
    # Process all a and b values on the right
    for i in range(len(all_right_a)):
        a = all_right_a[i]
        b = all_right_b[i]
        # Check if within bounds
        if is_in_bounds(a, right_bound_a) or is_in_bounds(b, right_bound_b):
            filtered_right_a.append(a) 
            filtered_right_b.append(b) 
          
    # Number of points filtered on both sides
    left_num  = len(filtered_left_a)
    right_num = len(filtered_right_a)
    
    """
    print ("Filtered left lines")
    for i in range(left_num):
        print(filtered_left_a[i], filtered_left_b[i])
        
    print ("Filtered right lines")
    for i in range(right_num):
        print(filtered_right_a[i], filtered_right_b[i])
    """
    
    # Sometimes it can happen that either left or right side produces no lines
    # In that case we will skip calculating the affected average line
    # Therefore, define these flags for success
    left_successful  = True
    right_successful = True
    
    if left_num > 0:
        left_avg_a  = sum(filtered_left_a)  / left_num
        left_avg_b  = sum(filtered_left_b)  / left_num
    else:
        left_avg_a, left_avg_b = 0, 0
        left_successful = False
        
    if right_num > 0:    
        right_avg_a = sum(filtered_right_a) / right_num
        right_avg_b = sum(filtered_right_b) / right_num
    else:
        right_avg_a, right_avg_b = 0, 0
        right_successful = False
    
    """
    print ("Found filtered size and averages on both sides")
    print (left_num, left_avg_a, left_avg_b, right_num, right_avg_a, right_avg_b)
    """
    
    # Final average lines
    avg_line_left, avg_line_right = [], []
    
    # For each average line, we need two points. 
    # We have slopes from the calculations above
    # We will define Y values to be on top and bottom of
    # the masking are defined with vertices. 
    # We can then calcualte X values by using slope and Y (x = (y - b)/a)
    left_line_p1_y  = vertices[0][0][1]
    left_line_p2_y  = vertices[0][1][1]
    right_line_p1_y = vertices[0][2][1]
    right_line_p2_y = vertices[0][3][1]
    
    # Calculate the corresponding X values, and finally lines
    
    if left_successful == True:
        left_line_p1_x  = (left_line_p1_y - left_avg_b)  / left_avg_a
        left_line_p2_x  = (left_line_p2_y - left_avg_b)  / left_avg_a
        avg_line_left = [[int(left_line_p1_x), int(left_line_p1_y), int(left_line_p2_x), int(left_line_p2_y)]]
        
    if right_successful == True:
        right_line_p1_x = (right_line_p1_y - right_avg_b) / right_avg_a
        right_line_p2_x = (right_line_p2_y - right_avg_b) / right_avg_a
        avg_line_right = [[int(right_line_p1_x), int(right_line_p1_y), int(right_line_p2_x), int(right_line_p2_y)]]
    
    return avg_line_left, avg_line_right

def combine_lines(lines, vertices):
    """
    Takes the list of lines, for example the output of Hough lines algorithm, 
    and produces one line (rectangle) for each side of the camera view (right and left).
    The approach is to calculate average a and b for left and right side, and take
    these values as a and b of the new lines. The final rectangles are produced by
    fattening the lines to the width of about 5-10 pixels. 
    """
    
    # The list vertices is specified to filter out the lines outside of 
    # the area of interest. We can use the mid point to understand which lines
    # belong to the left, and which to the right side
    vertices_mid = (vertices[0][1][0] + vertices[0][2][0])/2
    
    # First calculate the bounds for a and b on both sides of the field of view
    # These values will be used to filter out lines in the second pass
    all_left_a, all_left_b, all_right_a, all_right_b, left_bound_a, left_bound_b, right_bound_a, right_bound_b = \
        get_left_and_right_lines_bounds(lines, vertices_mid)

    # Calculate the average lines. Expected to get between 0 and 2 lines, mostly 2. 
    avg_line_left, avg_line_right = \
        get_average_lines(all_left_a, all_left_b, all_right_a, all_right_b, left_bound_a, left_bound_b, right_bound_a, right_bound_b, vertices)
     
    return avg_line_left, avg_line_right

def get_historical_avg_lines(avg_line_left, avg_line_right, left_past, right_past):
    """
    Calculates historical average over a number of frames, for average slope and
    intercept of the single lines on the left and right sides of the field of view. 
    The idea is that integrating over a historical number of frames (e.g. 5) will
    make the like smooth in videos and prevent large jumps between frames
    """
    # Add the current lines, but only if they are non-empty lists
    # The empty list means that no lines were collected
    if len(avg_line_left) > 0:
        left_past.append(avg_line_left)
    if len(avg_line_right) > 0:
        right_past.append(avg_line_right)

    # Check how many items do we have in the past
    # If more than threshold (e.g. 5), remove the oldest one
    num_left  = len(left_past)
    num_right = len(right_past)
    if num_left > 5:
        _ = left_past.popleft()
        num_left -= 1
    if num_right > 5:
        _ = right_past.popleft()
        num_right -= 1

    # These lists hold the average values for left and right average lines, averaged
    # over the kept history
    left_points, right_points = [[0, 0, 0, 0]], [[0, 0, 0, 0]]

    # Add all the existing points on the left side
    # left_points contains the sum of all the history points
    for i in range(num_left):
        curr_line = left_past[i]
        for j in range(len(curr_line[0])):
            left_points[0][j] += curr_line[0][j]

    # Add all the existing points on the right side
    # right_points contains the sum of all the history points
    for i in range(num_right):
        curr_line = right_past[i]
        for j in range(len(curr_line[0])):
            right_points[0][j] += curr_line[0][j]

    # Divide the lists with the actual number of points to get the historical average
    # The final lists need to be integers, hence int()
    for i in range(4):
        left_points[0][i]  = int(left_points[0][i]  / num_left)
        right_points[0][i] = int(right_points[0][i] / num_right)

    return [left_points, right_points]
            
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap, vertices, left_past=-1, right_past=-1):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    # Calculate Hough lines
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    
    # Combine Hough lines in a way that we get a single line on each 
    # side (left and right) of the field of view. These single lines are 
    # obtained by averaging Hough lines
    avg_line_left, avg_line_right = combine_lines(lines, vertices)
    
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    
    # Check if we need to average over previous frames. The idea is that by 
    # averaging over previous frames we achieve smoother line movements, 
    # preventing sudden jumps between frames
    if left_past == -1 and right_past == -1:
        # Historical averaging is disabled, just use the lines calculated
        lines_combined = [avg_line_left, avg_line_right]
        draw_lines(line_img, lines_combined, thickness=10)
    else:
        # Historical averaging is enabled
        # Create the final list lines_combined, which contains both left and 
        # right averaged lines. Draw the lines over the image. 
        lines_combined = get_historical_avg_lines(avg_line_left, avg_line_right, left_past, right_past)
        draw_lines(line_img, lines_combined, thickness=10)
        
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
os.listdir("test_images/")

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [ ]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images_output directory.

import math

def process_image_worker(img_orig):
    """
    Implements the processing steps for the image. The steps are
    implemented using the helper functions defined above
    """
    
    # Convert to grayscale
    img_gray = grayscale(img_orig)
    
    # Apply Gaussian blur before looking for edges
    img_blur = gaussian_blur(img_gray, 5)
    
    # Get Canny edges
    img_canny = canny(img_blur, 50, 150)
    
    # Define vertices for the region of interest, and filter out 
    # the edges with respect to that region
    vertices = np.array([[(150,540),(460, 320), (500, 320), (900,540)]], dtype=np.int32)
    img_roi = region_of_interest(img_canny, vertices)
    
    # Define parameters for the Hough transform 
    rho = 2             # distance resolution in pixels of the Hough grid
    theta = 1*np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 5       # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 10   # minimum number of pixels making up a line
    max_line_gap = 5    # maximum gap in pixels between connectable line segments
    
    # Get Hough lines 
    img_hough = hough_lines(img_roi, rho, theta, threshold, min_line_len, max_line_gap, vertices)
    
    # Add the Hough lines to the original image for display purposes
    img_w = weighted_img(img_hough, img_orig)
    
    return img_w

def pipeline(img_file_path):
    img_orig  = mpimg.imread(in_img_path)
    img_final = process_image_worker(img_orig)
    return img_final
    
for in_img in os.listdir("test_images/"):
    in_img_path = "test_images/" + in_img
    print("Processing input image: " + in_img_path)
    img_processed = pipeline(in_img_path)
    out_img_path = "test_images_output/" + in_img 
    mpimg.imsave(out_img_path, img_processed)
    # plt.imshow(img_processed)
    print("Created output image: " + out_img_path)

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
from collections import deque
left_past  = deque([])
right_past = deque([])
def process_image(img_orig):
    """
    Implements the processing steps for the image. The steps are
    implemented using the helper functions defined above
    """
    
    # Convert to grayscale
    img_gray = grayscale(img_orig)
    
    # Apply Gaussian blur before looking for edges
    img_blur = gaussian_blur(img_gray, 5)
    
    # Get Canny edges
    img_canny = canny(img_blur, 50, 150)
    
    # Define vertices for the region of interest, and filter out 
    # the edges with respect to that region
    vertices = np.array([[(150,540),(460, 320), (500, 320), (900,540)]], dtype=np.int32)
    img_roi = region_of_interest(img_canny, vertices)
    
    # Define parameters for the Hough transform 
    rho = 2             # distance resolution in pixels of the Hough grid
    theta = 1*np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 5       # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 10   # minimum number of pixels making up a line
    max_line_gap = 5    # maximum gap in pixels between connectable line segments
    
    # Get Hough lines 
    img_hough = hough_lines(img_roi, rho, theta, threshold, min_line_len, max_line_gap, vertices, left_past, right_past)
    
    # Add the Hough lines to the original image for display purposes
    img_w = weighted_img(img_hough, img_orig)
    
    return img_w

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))